In [2]:
import os
import glob
import numpy as np
import pprint
import functools

import tensorflow as tf

from ipywidgets import interact, interactive, fixed, interact_manual, HBox, VBox, Button
import ipywidgets as widgets

In [3]:
import sys
sys.executable

'/home/lcances/.miniconda3/envs/tensorboard/bin/python'

In [4]:
# global variables
tag_list = []
log_list = []
file_stats = dict()
stat_list = ["mean", "std", "maxi", "mini"]

os.chdir("../..")
tensorboard_root = "UrbanSound8k/tensorboard"

# Utility function

In [5]:
def get_all_tag(path):
    global tag_list
    tag_list = []

    for e in tf.compat.v1.train.summary_iterator(path):
        for value in e.summary.value:
            if value.HasField("simple_value"):
                if value.tag not in tag_list:
                    tag_list.append(value.tag)
    
    return tag_list

- Si la list n'est pas vide
    - Si le premier element est un fichier: return True
    - Si le premier element est un dossier:
        - lister le contenue de ce dossier L
        - si L n'est pas vide
            - Si le premier element est un fichier: return True
            - Sinon return False

In [6]:
def contain_log(path):
    if path:
        if os.path.isfile(path[0]):
            return True
        
        if os.path.isdir(path[0]):
            if not os.listdir(path[0]): # directory is empty
                return False
            
            sublist = [os.path.join(path[0], p) for p in os.listdir(path[0])]
            
            if sublist:
                if os.path.isfile(sublist[0]):
                    return True
                else:
                    return False
    return True

In [7]:
def get_tag_list(log_file):
    list_file = os.listdir(log_file)
    
    print(log_file)
    print(list_file)
    path = os.path.join(log_file, list_file[0])
    tag_list = get_all_tag(path)
    print(tag_list)

In [8]:
def ui_dropdown_dir(directory):
    global log_list
    
    sub_directories = sorted([os.path.join(directory, p) for p in os.listdir(directory) ])
        
    if not contain_log(sub_directories):
        interact(ui_dropdown_dir, directory=sub_directories)
        
    else:
        path_list = [os.path.join(directory, l) for l in os.listdir(directory)]
        log_list = path_list
        interact_manual(get_tag_list, log_file=path_list)

In [9]:
def stat_stat(stat = "maxi"):
    global file_stats
    
    values = [file_stats[l][stat] for l in file_stats.keys()]
    
    print("stat stat")
    print("mean: ", np.nanmean(values))
    print("std : ", np.nanstd(values))
    print("mini: ", np.nanmin(values))
    print("maxi: ", np.nanmax(values))
    print("")
    print("%.3f ± %.3f" % (np.nanmean(values), np.nanstd(values)))
    print("")
    print("detail ")
    pprint.pprint(values)

In [10]:
glob.glob(tensorboard_root + "/**")

['UrbanSound8k/tensorboard/deep-co-training_aug4adv',
 'UrbanSound8k/tensorboard/compound_scaling',
 'UrbanSound8k/tensorboard/full_supervised_test',
 'UrbanSound8k/tensorboard/deep-co-training',
 'UrbanSound8k/tensorboard/deep-co-training_noAdv',
 'UrbanSound8k/tensorboard/deep-co-training_independant-loss',
 'UrbanSound8k/tensorboard/full_supervised',
 'UrbanSound8k/tensorboard/script_tensorboard.Sh']

In [11]:
def ui_dropdown_dir(directory, filters: str = ""):
    global log_list
    
    sub_directories = sorted([os.path.join(directory, p) for p in os.listdir(directory) ])
        
    if not contain_log(sub_directories):
        print("next")
        ui = interact(ui_dropdown_dir, directory=sub_directories, filters="")
        
    else:
        valid_list, path_list = [], []
        
        # prepare filters
        if filters == "":
            filters = [""]
            need_filters = [""]
            avoid_filters = []
        else:
            filters = filters.split(",")
            need_filters = [f for f in filters if f[0] != "~"]
            avoid_filters = [f[1:] for f in filters if f[0] == "~"]
        if not need_filters:
            need_filters = [""]
            
        print("need filters: ", need_filters)
        print("avoid filters: ", avoid_filters)
        # need filters
        for l in os.listdir(directory):
            valid = 0
            for f in need_filters:
                if f in l:
                    valid += 1
                    
            if valid == len(need_filters):
                valid_list.append(l)
                    
        # avoid filters
        for l in valid_list:
            valid = True
            
            for f in avoid_filters:
                if f in l:
                    valid = False
                    break
                    
            if valid:
                path_list.append(os.path.join(directory, l))
            
        print("valid list: ", valid_list[:2])
        print("path list: ", path_list[:2])
        # path_list = [os.path.join(directory, l) for l in os.listdir(directory) if filters in l]
        log_list = path_list
        interact_manual(get_tag_list, log_file=path_list)

In [12]:
def stat_stat(stat = "maxi"):
    global file_stats
    
    values = [file_stats[l][stat] for l in file_stats.keys()]
    
    print("stat stat")
    print("mean: ", np.nanmean(values))
    print("std : ", np.nanstd(values))
    print("mini: ", np.nanmin(values))
    print("maxi: ", np.nanmax(values))
    print("")
    print("%.3f ± %.3f" % (np.nanmean(values) * 100, np.nanstd(values) * 100))
    print("")
    print("detail ")
    pprint.pprint(values)

In [13]:
def format_logs(stat = "maxi", format_fn = None):
    global file_stats
    
    msg = ""
    for k, d in file_stats.items():
        formated_name = k
        
        if format_fn is not None:
            formated_name = format_fn(k)
            
        msg += "%s %.6f\n" % (formated_name, d[stat])
    return msg.replace(".", ",")
            
    

In [18]:
def statistics(mean=True, std=True, max=False, min=False, tag=tag_list[0]):
    global file_stats
    
    file_log = dict()
    
    # read all the log files ------------------------------
    @functools.lru_cache()
    def get_log_data(path):
        logs = dict()

        for e in tf.compat.v1.train.summary_iterator(path):
            for value in e.summary.value:
                if value.HasField("simple_value"):
                    if value.tag not in logs:
                        logs[value.tag] = []
                    logs[value.tag].append(value.simple_value)
    
        return logs
    
    for l in log_list:
        final_path = os.path.join(l, os.listdir(l)[0])
        logs = get_log_data(final_path)
        
        if logs:
            file_log[l] = logs
        
    # compute the statistics ------------------------------
    # ---- file wise ----
    file_stats = dict()
    
    for l in file_log.keys():
        file_stats[l] = dict(
            mean = np.nanmean(file_log[l][tag]),
            std = np.nanstd(file_log[l][tag]),
            mini = np.nanmin(file_log[l][tag]),
            maxi = np.nanmax(file_log[l][tag]),
        )
        
    interact(stat_stat, stat=stat_list)

In [17]:
interact(ui_dropdown_dir, directory=glob.glob(tensorboard_root + "/**"), filters="")

interactive(children=(Dropdown(description='directory', options=('UrbanSound8k/tensorboard/deep-co-training_au…

<function __main__.ui_dropdown_dir(directory, filters:str='')>

In [19]:
interact_manual(statistics, mean=True, std=True, mini=False, maxi=False, tag=tag_list)

interactive(children=(Checkbox(value=True, description='mean'), Checkbox(value=True, description='std'), Check…

<function __main__.statistics(mean=True, std=True, max=False, min=False, tag='train/total_loss')>

In [30]:
def format_fn(n):
    print(n)
    base = n.split("/")[5].split("_")
#     print(base)
    
    model_name = base[2]
    ratio = base[3]
    epoch = base[4][:-1]
    lr = base[5][:-2]
    rule = base[6][:-4]
    nb_steps = base[7][:-1]
    nb_cycle = base[8][:-5]
    nb_beta = base[9][:-4]
    plsup_mini = base[10][:-1]
    
#     formated = "%s %s %s %s %s" % (model_name, lr, rule, nb_steps, epoch)
    formated = "%s %s %s %s %s %s %s %s" % (model_name, lr, rule, nb_steps, nb_cycle, nb_beta, plsup_mini, epoch)
    return formated

detail = format_logs("maxi", format_fn).split("\n")
detail = sorted(detail, key=lambda x: x.split(" ")[0])
print("\n".join(detail))

UrbanSound8k/tensorboard/deep-co-training_independant-loss/1lcm_1ldm/grid_search/2020-08-19_15:10:20_cnn03_0.1S_1000e_0.0005lr_linear-rfn_10s_1cycle_1beta_0.05m_plsup
UrbanSound8k/tensorboard/deep-co-training_independant-loss/1lcm_1ldm/grid_search/2020-08-19_15:13:45_cnn03_0.1S_1000e_0.0005lr_linear-rfn_10s_1cycle_1beta_0.1m_plsup
UrbanSound8k/tensorboard/deep-co-training_independant-loss/1lcm_1ldm/grid_search/2020-08-19_15:17:05_cnn03_0.1S_1000e_0.0005lr_linear-rfn_10s_1cycle_1beta_0.2m_plsup
UrbanSound8k/tensorboard/deep-co-training_independant-loss/1lcm_1ldm/grid_search/2020-08-19_15:20:52_cnn03_0.1S_1000e_0.0005lr_linear-rfn_10s_1cycle_1beta_0.3m_plsup
UrbanSound8k/tensorboard/deep-co-training_independant-loss/1lcm_1ldm/grid_search/2020-08-19_15:27:24_cnn03_0.1S_1000e_0.0005lr_linear-rfn_10s_1cycle_1beta_0.5m_plsup
UrbanSound8k/tensorboard/deep-co-training_independant-loss/1lcm_1ldm/grid_search/2020-08-19_15:24:12_cnn03_0.1S_1000e_0.0005lr_linear-rfn_10s_1cycle_1beta_0.4m_plsup
Urb

In [ ]:
def contain_log(path):
    if path:
        
        if os.path.isfile(path[0]):
            return True
        
        if os.path.isdir(path[0]):
            sublist = [os.path.join(path[0], p) for p in os.listdir(path[0])]
            
            if sublist:
                if os.path.isfile(sublist[0]):
                    return True
                else:
                    return False
    return True

In [ ]:
root = "/mnt/e/sync/Documents_sync/Projet/UrbanSound8K/tensorboard/osirim_tensorboard"
list_dir = os.listdir(root)

for name in list_dir:
    path = os.path.join(root, name)
    
    print(contain_log(path))

In [51]:
interact(ui_dropdown_dir, directory=glob.glob(tensorboard_root + "/**"))

interactive(children=(Dropdown(description='directory', options=(), value=None), Output()), _dom_classes=('wid…

<function __main__.ui_dropdown_dir(directory)>